In [ ]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

# Visualizing libraries
from matplotlib.pyplot import matplotlib as plt
import seaborn as sns
import plotly.express as px

In [ ]:
# Load the dataset both train and test
train = pd.read_csv('/content/drive/MyDrive/P6/Paitients_Files_Train.csv')
eval = pd.read_csv('/content/drive/MyDrive/P6/Paitients_Files_Test.csv')

In [ ]:
# # While reading in local machine
# train=pd.read_csv('C:\\Users\\user\\Downloads\\P6\\Paitients_Files_Train.csv')
# eval = pd.read_csv('C:\\Users\\user\\Downloads\\P6\\Paitients_Files_Test.csv')

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 599 entries, 0 to 598
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ID         599 non-null    object 
 1   PRG        599 non-null    int64  
 2   PL         599 non-null    int64  
 3   PR         599 non-null    int64  
 4   SK         599 non-null    int64  
 5   TS         599 non-null    int64  
 6   M11        599 non-null    float64
 7   BD2        599 non-null    float64
 8   Age        599 non-null    int64  
 9   Insurance  599 non-null    int64  
 10  Sepssis    599 non-null    object 
dtypes: float64(2), int64(7), object(2)
memory usage: 51.6+ KB


# BASIC EDA

In [ ]:
# Read into our datasets
train.head()

,ID,PRG,PL,PR,SK,TS,M11,BD2,Age,Insurance,Sepssis
0,ICU200010,6,148,72,35,0,33.6,0.627,50,0,Positive
1,ICU200011,1,85,66,29,0,26.6,0.351,31,0,Negative
2,ICU200012,8,183,64,0,0,23.3,0.672,32,1,Positive
3,ICU200013,1,89,66,23,94,28.1,0.167,21,1,Negative
4,ICU200014,0,137,40,35,168,43.1,2.288,33,1,Positive


In [ ]:
eval.head()

,ID,PRG,PL,PR,SK,TS,M11,BD2,Age,Insurance
0,ICU200609,1,109,38,18,120,23.1,0.407,26,1
1,ICU200610,1,108,88,19,0,27.1,0.400,24,1
2,ICU200611,6,96,0,0,0,23.7,0.190,28,1
3,ICU200612,1,124,74,36,0,27.8,0.100,30,1
4,ICU200613,7,150,78,29,126,35.2,0.692,54,0


**Renaming the columns for better understanding**

In [ ]:
col_name_1 = {
    'ID' : 'ID' ,
    'PRG' : 'PlasmaGlucose',
    'PL' : 'Blood.W.Result-1',
    'PR' : 'BloodPressure',
    'SK' : 'Blood.W.Result-2',
    'TS' : 'Blood.W.Result-3',
    'M11' : 'BMI',
    'BD2' : 'Blood.W.Result-4',
    'Age' : 'Age',
    'Insurance' : 'Insurance'
}

train.rename(columns=col_name_1, inplace= True)

In [ ]:
train.head(1)

,ID,PlasmaGlucose,Blood.W.Result-1,BloodPressure,Blood.W.Result-2,Blood.W.Result-3,BMI,Blood.W.Result-4,Age,Insurance,Sepssis
0,ICU200010,6,148,72,35,0,33.6,0.627,50,0,Positive


In [ ]:
eval.rename(columns=col_name_1, inplace= True)

In [ ]:
eval.head(1)

,ID,PlasmaGlucose,Blood.W.Result-1,BloodPressure,Blood.W.Result-2,Blood.W.Result-3,BMI,Blood.W.Result-4,Age,Insurance
0,ICU200609,1,109,38,18,120,23.1,0.407,26,1


**Check for missing values**

In [ ]:
train.isna().sum()

ID                  0
PlasmaGlucose       0
Blood.W.Result-1    0
BloodPressure       0
Blood.W.Result-2    0
Blood.W.Result-3    0
BMI                 0
Blood.W.Result-4    0
Age                 0
Insurance           0
Sepssis             0
dtype: int64

In [ ]:
eval.isna().sum()

ID                  0
PlasmaGlucose       0
Blood.W.Result-1    0
BloodPressure       0
Blood.W.Result-2    0
Blood.W.Result-3    0
BMI                 0
Blood.W.Result-4    0
Age                 0
Insurance           0
dtype: int64

In [ ]:
train.head()

,ID,PlasmaGlucose,Blood.W.Result-1,BloodPressure,Blood.W.Result-2,Blood.W.Result-3,BMI,Blood.W.Result-4,Age,Insurance,Sepssis
0,ICU200010,6,148,72,35,0,33.6,0.627,50,0,Positive
1,ICU200011,1,85,66,29,0,26.6,0.351,31,0,Negative
2,ICU200012,8,183,64,0,0,23.3,0.672,32,1,Positive
3,ICU200013,1,89,66,23,94,28.1,0.167,21,1,Negative
4,ICU200014,0,137,40,35,168,43.1,2.288,33,1,Positive


### **3) Identiying columns to drop**

Since our target is the sepsis medical condition , we will have to drop columns unassociated with the sepssis disease ,

In this scenario we will have to drop ID column and Insuarance column.
Before dropping the Insurance column, we will use a univariate analysis on it to see how it relates to other columns.

#### **a) sepsiss vs Insurance**

In [ ]:
insurance = train[['Insurance', 'Sepssis']]

insurance.value_counts()

Insurance  Sepssis 
1          Negative    260
           Positive    151
0          Negative    131
           Positive     57
dtype: int64

In [ ]:
k ={
    'Insurance' : ['Yes', 'Yes', 'No', 'No'],
    'Sepsssis' : ['Negative' ,'Positive' ,'Negative','Positive' ],
    'Frequency' : [260, 151, 131, 57]
}
k = pd.DataFrame(k)
k

,Insurance,Sepsssis,Frequency
0,Yes,Negative,260
1,Yes,Positive,151
2,No,Negative,131
3,No,Positive,57


In [ ]:
fig = px.bar( k , x='Insurance', y='Frequency',
             hover_data=['Insurance'], color='Sepsssis',
             labels={'pop':'frequency'}, height=400)
fig.show()

#### **b) Disease VS Age**

In [ ]:
train['Age'].nunique()

50

In [ ]:
train['Age'].min()

21

In [ ]:
train['Age'].max()

81

In [ ]:
#train['Age'].value_counts()

In [ ]:
age = train[['Age', 'Sepssis']]

In [ ]:
# # min 21 , Max 81
# # G1 = 21-35 , G2 = 35 - 45 , G3 = 46 - 60, G4 = Above 60
# age['21 - 35 yrs'] = age[['Age'] < 36]
# age['36 - 45 yrs'] = (age[['Age'] > 35]) & (age[['Age'] < 45])
# age['46 - 60 yrs'] = (age[['Age'] > 45]) & (age[['Age'] < 61])
# age['Above yrs'] = age[['Age'] > 60]

In [ ]:
age

,Age,Sepssis
0,50,Positive
1,31,Negative
2,32,Positive
3,21,Negative
4,33,Positive
...,...,...
594,34,Negative
595,22,Positive
596,46,Negative
597,21,Negative


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 599 entries, 0 to 598
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                599 non-null    object 
 1   PlasmaGlucose     599 non-null    int64  
 2   Blood.W.Result-1  599 non-null    int64  
 3   BloodPressure     599 non-null    int64  
 4   Blood.W.Result-2  599 non-null    int64  
 5   Blood.W.Result-3  599 non-null    int64  
 6   BMI               599 non-null    float64
 7   Blood.W.Result-4  599 non-null    float64
 8   Age               599 non-null    int64  
 9   Insurance         599 non-null    int64  
 10  Sepssis           599 non-null    object 
dtypes: float64(2), int64(7), object(2)
memory usage: 51.6+ KB


In [ ]:
train['ID']

0      ICU200010
1      ICU200011
2      ICU200012
3      ICU200013
4      ICU200014
         ...    
594    ICU200604
595    ICU200605
596    ICU200606
597    ICU200607
598    ICU200608
Name: ID, Length: 599, dtype: object

In [ ]:
# Dropping the ID column

In [ ]:
train.drop('ID', axis=True, inplace = True)

In [ ]:
#Importing Encoders and scalers
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

#Importing splitting module
from sklearn.model_selection import train_test_split

#Importing Models to run
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm

# Import warning filter
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train.head(2)

,PlasmaGlucose,Blood.W.Result-1,BloodPressure,Blood.W.Result-2,Blood.W.Result-3,BMI,Blood.W.Result-4,Age,Insurance,Sepssis
0,6,148,72,35,0,33.6,0.627,50,0,Positive
1,1,85,66,29,0,26.6,0.351,31,0,Negative


In [ ]:
scaler = StandardScaler()

In [ ]:
X = train.drop('Sepssis', axis = 1)
y = train['Sepssis']

In [ ]:
X.head(1)

,PlasmaGlucose,Blood.W.Result-1,BloodPressure,Blood.W.Result-2,Blood.W.Result-3,BMI,Blood.W.Result-4,Age,Insurance
0,6,148,72,35,0,33.6,0.627,50,0


In [ ]:
y.head(2)

0    Positive
1    Negative
Name: Sepssis, dtype: object

In [ ]:
X_scaled = scaler.fit_transform(X)

In [ ]:
X_scaled_ = pd.DataFrame(X_scaled)

X_scaled_

,0,1,2,3,4,5,6,7,8
0,0.647402,0.852744,0.169109,0.902098,-0.682191,0.209955,0.432333,1.413836,-1.478570
1,-0.840679,-1.076512,-0.141457,0.527197,-0.682191,-0.664876,-0.386002,-0.193804,-1.478570
2,1.242635,1.924553,-0.244979,-1.284822,-0.682191,-1.077297,0.565757,-0.109191,0.676329
3,-0.840679,-0.954020,-0.141457,0.152297,0.124823,-0.477412,-0.931558,-1.039930,0.676329
4,-1.138295,0.515890,-1.487246,0.902098,0.760132,1.397227,5.357163,-0.024579,0.676329
...,...,...,...,...,...,...,...,...,...
594,0.647402,0.087166,0.169109,1.526932,1.292417,0.209955,0.746620,0.060034,-1.478570
595,-1.138295,2.077668,0.686720,-0.410054,0.906081,0.009994,0.595406,-0.955317,0.676329
596,-1.138295,-1.627728,0.376154,-1.284822,-0.682191,1.672174,-0.851503,1.075385,0.676329
597,-0.840679,-0.954020,-2.315423,-0.097637,-0.467559,-0.514905,0.230714,-1.039930,-1.478570


In [ ]:
# encode the categorical column, the target column

In [ ]:
le = LabelEncoder()

y_encoded = le.fit_transform(y)

y_encoded_ = pd.DataFrame(y_encoded)

y_encoded_.head()

,0
0,1
1,0
2,1
3,0
4,1


In [ ]:
col = {'0' : 'Sepssis'}
y_encoded_.rename(col, axis = 1)

,0
0,1
1,0
2,1
3,0
4,1
...,...
594,0
595,1
596,0
597,0


In [ ]:
train_ready = pd.concat([X_scaled_, y_encoded_ ], axis=1)

In [ ]:
train_ready.head()

,0,1,2,3,4,5,6,7,8,0
0,0.647402,0.852744,0.169109,0.902098,-0.682191,0.209955,0.432333,1.413836,-1.478570,1
1,-0.840679,-1.076512,-0.141457,0.527197,-0.682191,-0.664876,-0.386002,-0.193804,-1.478570,0
2,1.242635,1.924553,-0.244979,-1.284822,-0.682191,-1.077297,0.565757,-0.109191,0.676329,1
3,-0.840679,-0.954020,-0.141457,0.152297,0.124823,-0.477412,-0.931558,-1.039930,0.676329,0
4,-1.138295,0.515890,-1.487246,0.902098,0.760132,1.397227,5.357163,-0.024579,0.676329,1


In [ ]:
cols2 = {
'0' : 'ID',
'1' : 'PlasmaGlucose',
'2' : 'Blood.W.Result-1',
'3' : 'BloodPressure',
'4' : 'Blood.W.Result-2',
'5' : 'Blood.W.Result-3',
'6' : 'BMI',
'7' : 'Blood.W.Result-4',
'8' : 'Age',
'0' : 'Insurance'
        }

In [ ]:
train_ready.rename(cols2, axis = 1)

,0,1,2,3,4,5,6,7,8,0
0,0.647402,0.852744,0.169109,0.902098,-0.682191,0.209955,0.432333,1.413836,-1.478570,1
1,-0.840679,-1.076512,-0.141457,0.527197,-0.682191,-0.664876,-0.386002,-0.193804,-1.478570,0
2,1.242635,1.924553,-0.244979,-1.284822,-0.682191,-1.077297,0.565757,-0.109191,0.676329,1
3,-0.840679,-0.954020,-0.141457,0.152297,0.124823,-0.477412,-0.931558,-1.039930,0.676329,0
4,-1.138295,0.515890,-1.487246,0.902098,0.760132,1.397227,5.357163,-0.024579,0.676329,1
...,...,...,...,...,...,...,...,...,...,...
594,0.647402,0.087166,0.169109,1.526932,1.292417,0.209955,0.746620,0.060034,-1.478570,0
595,-1.138295,2.077668,0.686720,-0.410054,0.906081,0.009994,0.595406,-0.955317,0.676329,1
596,-1.138295,-1.627728,0.376154,-1.284822,-0.682191,1.672174,-0.851503,1.075385,0.676329,0
597,-0.840679,-0.954020,-2.315423,-0.097637,-0.467559,-0.514905,0.230714,-1.039930,-1.478570,0


### **SELECTING TARGET AND FEATURES**

In [ ]:
# Select features using slicing
X_train = train_ready.iloc[:, :-1]

X_train.head()

,0,1,2,3,4,5,6,7,8
0,0.647402,0.852744,0.169109,0.902098,-0.682191,0.209955,0.432333,1.413836,-1.478570
1,-0.840679,-1.076512,-0.141457,0.527197,-0.682191,-0.664876,-0.386002,-0.193804,-1.478570
2,1.242635,1.924553,-0.244979,-1.284822,-0.682191,-1.077297,0.565757,-0.109191,0.676329
3,-0.840679,-0.954020,-0.141457,0.152297,0.124823,-0.477412,-0.931558,-1.039930,0.676329
4,-1.138295,0.515890,-1.487246,0.902098,0.760132,1.397227,5.357163,-0.024579,0.676329


In [ ]:
# Selecting the target using target
y_train = pd.DataFrame(train_ready.iloc[:, -1])

y_train.head()

,0
0,1
1,0
2,1
3,0
4,1


In [ ]:
x_train, x_test, y_train, y_test = train_test_split (X_train, y_train, test_size=0.2, random_state=42) # stratify = y_train)

In [ ]:
print("x_train shape", x_train.shape)
print('x_test shape', x_test.shape)
print('y_train shape', y_train.shape)
print('y_test shape', y_test.shape)

x_train shape (479, 9)
x_test shape (120, 9)
y_train shape (479, 1)
y_test shape (120, 1)


### **MODEL SELECTION, FITTING, PREDICTION, EVAL**

In [ ]:
# instance creation
logistic = LogisticRegression ()           # 1
knn = KNeighborsClassifier ()              # 2
decisiontree = DecisionTreeClassifier ()   # 3
random = RandomForestClassifier ()         # 4
gradient = GradientBoostingClassifier ()   # 5

In [ ]:
# fit all models to the train and eval
logistic.fit(x_train, y_train)
knn.fit(x_train, y_train)
decisiontree.fit(x_train, y_train)
random.fit(x_train, y_train)
gradient.fit(x_train, y_train)


GradientBoostingClassifier()

In [ ]:
# Make predictions
logistc_pred = (logistic.fit(x_train, y_train)).predict(x_test)
knn_pred = (knn.fit(x_train, y_train)).predict(x_test)
decisiontree_pred = (decisiontree.fit(x_train, y_train)).predict(x_test)
random_pred = (random.fit(x_train, y_train)).predict(x_test)
gradient_pred = (gradient.fit(x_train, y_train)).predict(x_test)


### Make Evaluations to select the best model

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_squared_log_error,mean_absolute_error, accuracy_score

In [ ]:
# for logistic
r21 = r2_score( y_test, logistc_pred)
mse1 =mean_squared_error( y_test, logistc_pred)
mae1 = mean_absolute_error( y_test, logistc_pred)
accuracy1 = accuracy_score(y_test, logistc_pred)

# for knn
r22 = r2_score( y_test, knn_pred)
mse2 =mean_squared_error( y_test, knn_pred)
mae2 = mean_absolute_error( y_test, knn_pred)
accuracy2 = accuracy_score(y_test, knn_pred)

# for decision tree
r23 = r2_score( y_test, decisiontree_pred)
mse3 =mean_squared_error( y_test, decisiontree_pred)
mae3 = mean_absolute_error( y_test, decisiontree_pred)
accuracy3 = accuracy_score(y_test, decisiontree_pred)

# for random
r24 = r2_score( y_test, random_pred)
mse4 =mean_squared_error( y_test, random_pred)
mae4 = mean_absolute_error( y_test, random_pred)
accuracy4 = accuracy_score(y_test, random_pred)

# gradient
r25 = r2_score( y_test, gradient_pred)
mse5 =mean_squared_error( y_test, gradient_pred)
mae5 = mean_absolute_error( y_test, gradient_pred)
accuracy5 = accuracy_score(y_test, gradient_pred)

In [ ]:
metrics1 = pd.DataFrame ([['Linear_Regression', r21, mse1, mae1, accuracy1]], columns=['model', 'R2_SCORE', 'MSE', 'MAE', 'accuracy'])
metrics2 = pd.DataFrame ([['Knn', r22, mse2, mae2, accuracy2]], columns=['model', 'R2_SCORE', 'MSE', 'MAE', 'accuracy'])
metrics3 = pd.DataFrame ([['decision tree', r23, mse3, mae3, accuracy3]], columns=['model', 'R2_SCORE', 'MSE', 'MAE', 'accuracy'])
metrics4 = pd.DataFrame ([['Random forest', r24, mse4, mae4, accuracy4]], columns=['model', 'R2_SCORE', 'MSE', 'MAE', 'accuracy'])
metrics5 = pd.DataFrame ([['Gradient', r25, mse5, mae5, accuracy5]], columns=['model', 'R2_SCORE', 'MSE', 'MAE', 'accuracy'])

metrics = metrics1.append([metrics2, metrics3, metrics4, metrics5], ignore_index=True)

metrics

,model,R2_SCORE,MSE,MAE,accuracy
0,Linear_Regression,-0.123528,0.258333,0.258333,0.741667
1,Knn,-0.485956,0.341667,0.341667,0.658333
2,decision tree,-0.232256,0.283333,0.283333,0.716667
3,Random forest,-0.268499,0.291667,0.291667,0.708333
4,Gradient,-0.014799,0.233333,0.233333,0.766667


### **SAVE THE BEST MODEL I.e, Gradient Boosting**

In [153]:
model = gradient.fit(x_train, y_train)

In [190]:
import joblib

In [191]:
model_filename = "trained_model.joblib"
joblib.dump(model, model_filename)

['trained_model.joblib']

In [192]:
from google.colab import files
files.download(model_filename)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### **CONVERT TO PIPELINE**

Steps


*   Select from the dataframe,  x = (df.iloc[:, :-1]) and y=(df.iloc[:, -1])
*   Scale x features using standard scaler instance , use label encoder on y target
*   split the datasets, into train test and eval.  x_train, x_test, y_train, y_test = train_test_split (X_train, y_train, test_size=0.2, random_state=42)
*   Fit a gradient boosting model



In [167]:
df = pd.read_csv('/content/drive/MyDrive/P6/Paitients_Files_Train.csv')

In [172]:
df.drop('ID', axis=True, inplace = True)

In [173]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [184]:
from sklearn.pipeline import Pipeline

# Select x and y
x = df.iloc[:, :-1]
y = df.iloc[:, -1]

# Step 2: Scale x features using StandardScaler and encode y target using LabelEncoder
scaler = StandardScaler()
label_encoder = LabelEncoder()

x_scaled = scaler.fit_transform(x)
y_encoded = label_encoder.fit_transform(y)

# Step 3: Split the datasets into train, test, and eval sets
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y_encoded, test_size=0.2, random_state=42)

# Step 4: Fit a gradient boosting model
gradient_boosting_model = GradientBoostingClassifier()

# Step 5: Build the pipeline
pipeline = Pipeline([
    ('scaler', scaler),
    ('label_encoder', label_encoder),
    ('model', gradient_boosting_model)
])




In [189]:
pipeline

Pipeline(steps=[('scaler', StandardScaler()),
                ('label_encoder', LabelEncoder())])

In [186]:
pipeline = Pipeline([
    ('scaler', scaler),
    ('label_encoder', label_encoder)
])

In [181]:
x_train = pd.DataFrame(x_train)
y_train = pd.DataFrame(y_train)

In [182]:
# Step 6: Fit the pipeline on the training data
pipeline.fit(x_train, y_train)

TypeError: ignored